# PROJEKT MODELU KLASYFIKACYJNEGO PRZEWIDYWANIA STANU MIESZKAŃ NA PODSTAWIE #

## Extra Trees Classifier ##

### maj 2025

In [1]:
import pandas as pd
import mlflow
from pycaret.classification import setup, pull, compare_models, plot_model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import resample
import pymysql
from sqlalchemy import create_engine
import numpy as np
from scipy.stats import skewnorm
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# tą komórkę uruchom jeżeli bazę danych bierzesz np. z DBeaver, oracle developer itp.

#username = 'root'
#password = '1234'
#host = '127.0.0.1'
#port = 3306  
#database = 'projekt1'
#engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

#df = pd.read_sql("SELECT * FROM saleflats", con=engine)

#engine.dispose()

In [3]:
MLFLOW_EXPERIMENT_NAME = 'Investoro_Stan_Mieszkania'
MLFLOW_TAGS = {'data': 'Investoro_stany', 'library': 'pycaret'}

mlflow.set_tracking_uri("http://localhost:5000")

In [4]:
# tę komórkę uruchom jeśli czerpiesz dane z pliku .csv
df = pd.read_csv('data.csv', sep=',')

In [5]:
df

,SaleId,OriginalId,PortalId,Title,Description,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,...,Archive,Location,VoivodeshipNumber,CountyNumber,CommunityNumber,KindNumber,RegionNumber,SubRegionNumber,StreetNumber,EncryptedId
0,88,NaN,14,Mieszkanie trzypokojowe na sprzedaż,Mieszkanie o powierzchni 73m2 znajduje się na ...,73.00,766500.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Bojary",20.0,61.0,1.0,1.0,922490.0,NaN,NaN,02TgImo1wm-Ra4zUQxoPBw==
1,99,NaN,19,Sprzedam mieszkanie na parterze 64.8m2 Białyst...,Sprzedam mieszkanie na parterze 64.8m2 w 3-pię...,64.80,540000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Antoniuk",20.0,61.0,1.0,1.0,922426.0,NaN,NaN,D-NUdVbtN3_kIW7ExgXkvA==
2,115,NaN,3,"Mieszkanie bezczynszowe, 3 pokoje, 2 łazienki",SPRZEDAŻ WYŁĄCZNIE BEZPOŚREDNIA. Agencjom nier...,51.00,540000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Bagnówka",20.0,61.0,1.0,1.0,922461.0,NaN,NaN,lmBSU1XiXLSLOq1yKSe2kA==
3,140,NaN,14,Mieszkanie trzypokojowe na sprzedaż,***Oferta bez prowizji biura i podatku PCC!***...,67.62,544000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Ul. Józefa Karola Puchal...",20.0,61.0,1.0,1.0,922410.0,NaN,43141.0,HWachf5gb6Vq-sFYH4YWnA==
4,145,NaN,3,3 Pokoje- 48M2-Osiedle Dziesięciny,Przedstawiamy na sprzedaż 3 pokojowe mieszkani...,48.00,459000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Ul. Dziesięciny",20.0,61.0,1.0,1.0,922410.0,NaN,4656.0,AWsmt4ZY0X7lNv-6fSCn9Q==
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133882,4418401,NaN,2,Mieszkanie na sprzedaż - 59 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,59.99,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYftiMRU9ErLfA6XOKdBw9jY=
1133883,4418402,NaN,2,Mieszkanie na sprzedaż - 41 m² - 2 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,41.23,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYa7D8z7yjun2jGP_aZIReN8=
1133884,4418404,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,49.09,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYf9eM73_U8S05NBjfLiez6E=
1133885,4418405,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,49.48,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYRXVZdKE7iJxxFPu0Bq0ikc=


In [6]:
# pobranie próbki 10 losowych wierszy
df.sample(10)

,SaleId,OriginalId,PortalId,Title,Description,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,...,Archive,Location,VoivodeshipNumber,CountyNumber,CommunityNumber,KindNumber,RegionNumber,SubRegionNumber,StreetNumber,EncryptedId
505999,2895733,NaN,11,"Mieszkanie, Warszawa, Ochota, 49 m²","Mieszkanie, Warszawa, Ochota, 49 m² Ogłoszenio...",48.70,849000.0,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Ochota",14.0,65.0,6.0,8.0,NaN,NaN,NaN,bTW-BTvDkbZheEoOSgDOiAAsOdtEx9lpXOO_vaDANMA=
1042189,4246105,NaN,3,Osiedle Poetów | mieszkanie A_72,Przedmiotem ogłoszenia jest 2-pokojowe mieszka...,37.31,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Wołomiński, Radzymin, Ul. Juliusz...",14.0,34.0,9.0,4.0,921591.0,NaN,20291.0,xwIxRC2P_cX9s-GF2a5tURXVZdKE7iJxxFPu0Bq0ikc=
183468,1323983,NaN,11,"Mieszkanie, Kraków, Prądnik Biały, 45 m²","Mieszkanie, Kraków, Prądnik Biały, 45 m² Ogłos...",44.50,679000.0,NaN,NaN,NaN,...,NaN,"Małopolskie, Kraków",12.0,61.0,1.0,1.0,950463.0,NaN,NaN,MM-sYeJlsj9w6qAqneVYy8c0fBXCdCR2219soBArV2g=
72932,444553,NaN,11,"Mieszkanie, Poznań, Rataje, 50 m²","Mieszkanie, Poznań, Rataje, 50 m² Ogłoszenioda...",50.00,549000.0,NaN,NaN,NaN,...,NaN,"Wielkopolskie, Poznań",30.0,64.0,4.0,9.0,970081.0,NaN,NaN,XMyMFzYPmHDvaHyO5PWe7CtYZcYFk_Hc7yDaIrhDAiQ=
421049,2731501,NaN,18,"2 pok, 36,18m2, Nowa Huta po remoncie do wejscia",KRAKÓW NOWA HUTA REJON OS.KAZIMIERZOWSKIE 2 PO...,36.18,610000.0,NaN,NaN,NaN,...,NaN,"Małopolskie, Kraków, Kraków-nowa Huta, Osiedle...",12.0,61.0,3.0,9.0,950718.0,NaN,52856.0,YknpjMQE3cX0ExGj2A3yi_tiMRU9ErLfA6XOKdBw9jY=
709067,3376899,NaN,19,Mieszkanie Apartament Prądnik Bialy Rezydencja...,",,Royal Green'' Apartament znajdujący się w ,,...",40.00,900000.0,NaN,NaN,NaN,...,NaN,"Małopolskie, Kraków, Kraków-krowodrza, Prądnik...",12.0,61.0,2.0,9.0,950635.0,NaN,NaN,_LEt80l2sT5SK4HnV2B0qJgtwTGeq42piasdfG4vjEE=
778618,3571542,NaN,19,3 pokojowe + dodatkowy pokoj na biuro Kamienica,Witam Prowizję od sprzedaży mieszkania ponosi ...,73.00,459900.0,NaN,NaN,NaN,...,NaN,"Lubuskie, Żarski, Żary",8.0,11.0,2.0,1.0,988684.0,NaN,NaN,AiH2-k0g5XXwyfaHOL-2wOq7Mn4f0k3b73TvPoybyXk=
84122,519044,NaN,3,"Sprzedam mieszkanie o powierz.45, 87 m kw.",Piłsudskiego - na sprzedaż mieszkanie o powier...,45.87,195000.0,NaN,NaN,NaN,...,NaN,"Wielkopolskie, Ostrowski, Ostrów Wielkopolski",30.0,17.0,1.0,1.0,937132.0,NaN,NaN,onbsEoz10hSwHncmZNYPcWWNmS6N8JNuQIXxaNhB83k=
1020266,4200128,NaN,3,Mieszkanie Zielone Błonia Rumia Etap III,Zielone Błonia III to trzeci etap przedsięwzię...,53.26,459000.0,466754.0,592645.0,469000.0,...,NaN,"Pomorskie, Wejherowski, Rumia, Ul. Spacerowa",22.0,15.0,2.0,1.0,934694.0,NaN,20641.0,gclpWJpjMClxfjczfKFKSI_bCM8lJ876moazm9h2OAo=
453229,2796568,NaN,3,Nowe Promenady | mieszkanie 3-pok. | A2.5.2.M8,3-pokojowe mieszkanie numer A2.5.2.M8 na 2. pi...,60.71,NaN,NaN,NaN,NaN,...,NaN,"Dolnośląskie, Wrocław, Wrocław-psie Pole, Ul. ...",2.0,64.0,4.0,9.0,986745.0,NaN,12964.0,VsL3pr2wpya0Ge8Rs90azatLCFuALKtSKAVt1gPg8yU=


In [7]:
print(df.columns)

Index(['SaleId', 'OriginalId', 'PortalId', 'Title', 'Description', 'Area',
       'Price', 'OfferPrice', 'RealPriceAfterRenovation', 'OriginalPrice',
       'PricePerSquareMeter', 'NumberOfRooms', 'BuiltYear', 'Type',
       'BuildingType', 'BuildingCondition', 'OfferFrom', 'Floor', 'Floors',
       'TypeOfMarket', 'OwnerType', 'DateAddedToDatabase', 'DateAdded',
       'DateLastModification', 'DateLastRaises', 'NewestDate', 'AvailableFrom',
       'Link', 'Phone', 'MainImage', 'OtherImages', 'NumberOfDuplicates',
       'NumberOfRaises', 'NumberOfModifications', 'IsDuplicatePriceLower',
       'IsDuplicatePrivateOwner', 'Score', 'ScorePrecision', 'CommunityScore',
       'NumberOfCommunityComments', 'NumberOfCommunityOpinions', 'Archive',
       'Location', 'VoivodeshipNumber', 'CountyNumber', 'CommunityNumber',
       'KindNumber', 'RegionNumber', 'SubRegionNumber', 'StreetNumber',
       'EncryptedId'],
      dtype='object')


In [8]:
df['BuiltYear'] = pd.to_datetime(df['BuiltYear'], format='%Y', errors='coerce')

In [9]:
df_a = df.dropna(subset=['Description'])

In [10]:
df_a.isnull().sum()

SaleId                             0
OriginalId                   1125223
PortalId                           0
Title                              0
Description                        0
Area                            2077
Price                          53212
OfferPrice                   1043154
RealPriceAfterRenovation     1043154
OriginalPrice                1015114
PricePerSquareMeter            56195
NumberOfRooms                  13525
BuiltYear                     379005
Type                               0
BuildingType                  202788
BuildingCondition            1048333
OfferFrom                       5030
Floor                          56904
Floors                        499074
TypeOfMarket                   31709
OwnerType                     540943
DateAddedToDatabase                0
DateAdded                          0
DateLastModification          880033
DateLastRaises                827954
NewestDate                         0
AvailableFrom                1125223
L

In [11]:
df_b = df_a.dropna(subset=['Location'])

In [12]:
df_b.isnull().sum()

SaleId                             0
OriginalId                   1124683
PortalId                           0
Title                              0
Description                        0
Area                            2071
Price                          53153
OfferPrice                   1042614
RealPriceAfterRenovation     1042614
OriginalPrice                1014574
PricePerSquareMeter            56133
NumberOfRooms                  13494
BuiltYear                     378808
Type                               0
BuildingType                  202543
BuildingCondition            1047795
OfferFrom                       5030
Floor                          56828
Floors                        498541
TypeOfMarket                   31629
OwnerType                     540441
DateAddedToDatabase                0
DateAdded                          0
DateLastModification          879543
DateLastRaises                827486
NewestDate                         0
AvailableFrom                1124683
L

In [13]:
df_c = df_b.dropna(subset=['BuildingCondition'])

In [14]:
df_c.isnull().sum()

SaleId                           0
OriginalId                   76888
PortalId                         0
Title                            0
Description                      0
Area                            30
Price                         7036
OfferPrice                   31258
RealPriceAfterRenovation     31258
OriginalPrice                73440
PricePerSquareMeter           7138
NumberOfRooms                  165
BuiltYear                    19248
Type                             0
BuildingType                 15600
BuildingCondition                0
OfferFrom                        0
Floor                         1984
Floors                       13600
TypeOfMarket                   716
OwnerType                    30525
DateAddedToDatabase              0
DateAdded                        0
DateLastModification         68857
DateLastRaises               68330
NewestDate                       0
AvailableFrom                76888
Link                             0
Phone               

In [15]:
df_c

,SaleId,OriginalId,PortalId,Title,Description,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,...,Archive,Location,VoivodeshipNumber,CountyNumber,CommunityNumber,KindNumber,RegionNumber,SubRegionNumber,StreetNumber,EncryptedId
1037441,4236909,NaN,10,Mieszkanie na sprzedaż - 56 m² - 3 pokoje Wars...,NAJBARDZIEJ ZIELONA DZIELNICA Białołęka ma now...,56.52,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Białołęka, Grodzisk, Ul...",14.0,65.0,3.0,8.0,919447.0,NaN,1051.0,YNwIfRAf_6pTlpT4SzWIF1I945vZrxCmGc1kx-MElW4=
1037442,4236910,NaN,10,Mieszkanie na sprzedaż - 39 m² - 3 pokoje Wars...,NAJBARDZIEJ ZIELONA DZIELNICA Białołęka ma now...,39.44,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Białołęka, Grodzisk, Ul...",14.0,65.0,3.0,8.0,919447.0,NaN,1051.0,YNwIfRAf_6pTlpT4SzWIF2b3MhONsWHx4CM5o0osslE=
1037444,4236917,NaN,10,Mieszkanie na sprzedaż - 57 m² - 4 pokoje Wars...,NAJBARDZIEJ ZIELONA DZIELNICA Białołęka ma now...,57.72,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Białołęka, Grodzisk, Ul...",14.0,65.0,3.0,8.0,919447.0,NaN,1051.0,YNwIfRAf_6pTlpT4SzWIF5P7_4V7ShzJmmF8PHTVfng=
1037449,4236925,NaN,10,Mieszkanie na sprzedaż - 34 m² - 2 pokoje Wars...,NAJBARDZIEJ ZIELONA DZIELNICA Białołęka ma now...,34.69,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Białołęka, Grodzisk, Ul...",14.0,65.0,3.0,8.0,919447.0,NaN,1051.0,YNwIfRAf_6pTlpT4SzWIF-baRshu3omO9IJePLIRnwA=
1037450,4236928,NaN,10,Mieszkanie na sprzedaż - 37 m² - 2 pokoje Wars...,NAJBARDZIEJ ZIELONA DZIELNICA Białołęka ma now...,37.03,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Warszawa, Białołęka, Grodzisk, Ul...",14.0,65.0,3.0,8.0,919447.0,NaN,1051.0,YNwIfRAf_6pTlpT4SzWIF4_bCM8lJ876moazm9h2OAo=
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133882,4418401,NaN,2,Mieszkanie na sprzedaż - 59 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,59.99,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYftiMRU9ErLfA6XOKdBw9jY=
1133883,4418402,NaN,2,Mieszkanie na sprzedaż - 41 m² - 2 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,41.23,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYa7D8z7yjun2jGP_aZIReN8=
1133884,4418404,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,49.09,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYf9eM73_U8S05NBjfLiez6E=
1133885,4418405,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,49.48,NaN,NaN,NaN,NaN,...,NaN,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYRXVZdKE7iJxxFPu0Bq0ikc=


In [16]:
df_c['Description'] = df_c['Description'].str.slice(0, 3000)

In [17]:
# Przygotuj wektorizer z limitem
vectorizer = CountVectorizer(max_features=1500)  # np. 1500 najczęstszych słów
X_bow = vectorizer.fit_transform(df_c["Description"])

# Konwertuj do DataFrame
df_bow = pd.DataFrame(X_bow.toarray(), columns=vectorizer.get_feature_names_out())

# Dołącz do oryginalnych danych (z wyjątkiem kolumny Description)
df_c = df_c.drop(columns=['Description']).reset_index(drop=True)
df_bow = df_bow.reset_index(drop=True)
df_prepared = pd.concat([df_c, df_bow], axis=1)

In [18]:
df_prepared.info

<bound method DataFrame.info of         SaleId  OriginalId  PortalId  \
0      4236909         NaN        10   
1      4236910         NaN        10   
2      4236917         NaN        10   
3      4236925         NaN        10   
4      4236928         NaN        10   
...        ...         ...       ...   
76883  4418401         NaN         2   
76884  4418402         NaN         2   
76885  4418404         NaN         2   
76886  4418405         NaN         2   
76887  4418406         NaN         2   

                                                   Title   Area  Price  \
0      Mieszkanie na sprzedaż - 56 m² - 3 pokoje Wars...  56.52    NaN   
1      Mieszkanie na sprzedaż - 39 m² - 3 pokoje Wars...  39.44    NaN   
2      Mieszkanie na sprzedaż - 57 m² - 4 pokoje Wars...  57.72    NaN   
3      Mieszkanie na sprzedaż - 34 m² - 2 pokoje Wars...  34.69    NaN   
4      Mieszkanie na sprzedaż - 37 m² - 2 pokoje Wars...  37.03    NaN   
...                                        

In [19]:
df_prepared[df_prepared.duplicated()]

,SaleId,OriginalId,PortalId,Title,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,PricePerSquareMeter,...,świeżo,świeżym,żabka,żadnych,że,życia,życie,życiem,żłobek,żłobki


In [20]:
df_prepared.nunique()

SaleId        76888
OriginalId        0
PortalId         34
Title         56154
Area           8156
              ...  
życia             8
życie             5
życiem            3
żłobek            5
żłobki            4
Length: 1550, dtype: int64

In [21]:
df_prepared.isnull()

,SaleId,OriginalId,PortalId,Title,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,PricePerSquareMeter,...,świeżo,świeżym,żabka,żadnych,że,życia,życie,życiem,żłobek,żłobki
0,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
3,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76883,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
76884,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
76885,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
76886,False,True,False,False,False,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False


In [22]:
df_prepared.isnull().sum()

SaleId            0
OriginalId    76888
PortalId          0
Title             0
Area             30
              ...  
życia             0
życie             0
życiem            0
żłobek            0
żłobki            0
Length: 1550, dtype: int64

In [23]:
Q1 = df_prepared["Price"].quantile(0.25)
Q3 = df_prepared["Price"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_prep_p = df_prepared[~((df_prepared["Price"] < lower_bound) | (df_prepared["Price"] > upper_bound))]

In [24]:
Q1 = df_prep_p["PricePerSquareMeter"].quantile(0.25)
Q3 = df_prep_p["PricePerSquareMeter"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_prep_a = df_prep_p[~((df_prep_p["PricePerSquareMeter"] < lower_bound) | (df_prep_p["PricePerSquareMeter"] > upper_bound))]

In [25]:
Q1 = df_prep_a["Area"].quantile(0.25)
Q3 = df_prep_a["Area"].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_prepared2 = df_prep_a[~((df_prep_a["Area"] < lower_bound) | (df_prep_a["Area"] > upper_bound))]

In [26]:
df_prepared2["BuildingCondition"].value_counts(normalize=True)

BuildingCondition
DEVELOPER_STATE     0.475638
AFTER_RENOVATION    0.349426
GOOD                0.087849
FOR_RENOVATION      0.087087
Name: proportion, dtype: float64

In [27]:
df_prepared2.nunique()

SaleId        69597
OriginalId        0
PortalId         34
Title         50634
Area           6522
              ...  
życia             8
życie             5
życiem            3
żłobek            5
żłobki            4
Length: 1550, dtype: int64

In [28]:
unique_btype=df_prepared2['BuildingType'].unique()

unique_btype

array([nan, 'Apartamentowiec', 'Blok', 'Pozostałe', 'Kamienica'],
      dtype=object)

In [29]:
print(df_prepared2['BuildingCondition'].nunique())

4


In [30]:
df_prepared2.isnull().sum()

SaleId            0
OriginalId    69597
PortalId          0
Title             0
Area             29
              ...  
życia             0
życie             0
życiem            0
żłobek            0
żłobki            0
Length: 1550, dtype: int64

In [31]:
df_prepared3=df_prepared2.dropna(subset=['BuildingCondition'])

In [32]:
df_prepared3.loc[df_prepared3['TypeOfMarket'] == 'pierwotny', 'BuildingCondition'] = 'DEVELOPER_STATE'

In [33]:
df_prepared3['BuiltYear'] = pd.to_datetime(df_prepared3['BuiltYear'], format='%Y', errors='coerce')

In [34]:
df_prepared3.loc[df_prepared3['BuiltYear'].dt.year >= 2024, 'BuildingCondition'] = 'DEVELOPER_STATE'

In [35]:
df_prepared3 = df_prepared3[~((df_prepared3['Link'].str.contains('otodom', case=False, na=False)) & 
                              (df_prepared3['BuildingCondition'] == 'AFTER_RENOVATION'))]

In [36]:
unique_ctype=df_prepared3['BuildingCondition'].unique()

unique_ctype

array(['DEVELOPER_STATE', 'AFTER_RENOVATION', 'GOOD', 'FOR_RENOVATION'],
      dtype=object)

In [37]:
df_prepared3["BuildingCondition"].value_counts(normalize=True)

BuildingCondition
DEVELOPER_STATE     0.618903
AFTER_RENOVATION    0.178385
FOR_RENOVATION      0.107558
GOOD                0.095154
Name: proportion, dtype: float64

In [38]:
print("Rozpoczynam ręczne balansowanie klas przed setup...")
df_to_balance = df_prepared3.copy() # Użyj df_prepared3 po wszystkich czyszczeniach

# Sprawdź ile rekordów ma każda klasa
class_counts = df_to_balance['BuildingCondition'].value_counts()
print("Liczebność klas przed zbalansowaniem:")
print(class_counts)

# Znajdź liczebność klasy większościowej, jeśli chcemy robić oversampling do niej,
# lub mniejszościowej, jeśli chcemy robić undersampling. SMOTE robi oversampling.
# Dla SMOTE z imblearn, nie musimy ręcznie robić resample, tylko przekazać dane do SMOTE.

from imblearn.over_sampling import SMOTE

# Przygotuj dane dla SMOTE
X_to_balance = df_to_balance.drop(columns=['BuildingCondition'])
y_to_balance = df_to_balance['BuildingCondition']

# Sprawdzenie, czy wszystkie kolumny w X_to_balance są numeryczne lub mogą być przekonwertowane.
# SMOTE nie działa dobrze z danymi kategorycznymi w formie stringów.
# W Twoim df_prepared3 masz kolumny liczbowe (bag-of-words) i kilka innych.
# Te inne (np. Location, BuildingType) muszą być odpowiednio zakodowane PRZED SMOTE.
# PyCaret robi to automatycznie, ale jeśli robimy SMOTE ręcznie, musimy to zrobić sami.
# To komplikuje sprawę.

# Alternatywnie, dla prostoty, możemy spróbować prostego resamplingu (jak było w Twoim kodzie)
# do liczby najmniejszej klasy (undersampling) lub do największej (oversampling z replace=True).
# SMOTE jest lepszy, ale wymaga bardziej ostrożnego przygotowania danych.

# Wróćmy do oryginalnego pomysłu z resample dla prostoty, jeśli SMOTE ręczny jest zbyt skomplikowany:
min_count = class_counts.min() # dla undersamplingu
# lub:
# target_count = class_counts.max() # dla oversamplingu
# lub ustal konkretną liczbę, np. średnią, medianę.

print(f"Balansowanie klas przez undersampling do: {min_count} próbek na klasę.")
dfs_balanced_list = []
for condition_class in class_counts.index:
    df_condition = df_to_balance[df_to_balance['BuildingCondition'] == condition_class]
    df_condition_resampled = resample(df_condition,
                                    replace=False, # False dla undersamplingu
                                    n_samples=min_count, # lub target_count z replace=True dla oversamplingu
                                    random_state=42)
    dfs_balanced_list.append(df_condition_resampled)

df_balanced_for_setup = pd.concat(dfs_balanced_list).reset_index(drop=True)
print("\nLiczebność klas PO zbalansowaniu:")
print(df_balanced_for_setup['BuildingCondition'].value_counts())
print(f"Kształt df_balanced_for_setup: {df_balanced_for_setup.shape}")

Rozpoczynam ręczne balansowanie klas przed setup...
Liczebność klas przed zbalansowaniem:
BuildingCondition
DEVELOPER_STATE     34778
AFTER_RENOVATION    10024
FOR_RENOVATION       6044
GOOD                 5347
Name: count, dtype: int64
Balansowanie klas przez undersampling do: 5347 próbek na klasę.

Liczebność klas PO zbalansowaniu:
BuildingCondition
DEVELOPER_STATE     5347
AFTER_RENOVATION    5347
FOR_RENOVATION      5347
GOOD                5347
Name: count, dtype: int64
Kształt df_balanced_for_setup: (21388, 1550)


In [39]:
# df_balanced["BuildingCondition"].value_counts(normalize=True)

In [40]:
df_balanced_for_setup

,SaleId,OriginalId,PortalId,Title,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,PricePerSquareMeter,...,świeżo,świeżym,żabka,żadnych,że,życia,życie,życiem,żłobek,żłobki
0,4318960,NaN,11,"Mieszkanie, Gdańsk, Stogi, 62 m²",62.05,729900.0,NaN,NaN,NaN,11763.09,...,0,0,0,0,0,0,0,0,0,0
1,4289189,NaN,18,Nowe apartamenty blisko Gdańska - Sprawdź,68.71,1117000.0,NaN,NaN,NaN,16256.73,...,0,1,0,0,2,0,0,0,0,0
2,4389971,NaN,3,"Nowe 2pok_Gotowe na IX 25""❗️Biuro Dewelopera❗️...",40.00,323915.0,318171.0,3.282760e+05,NaN,8097.88,...,0,0,0,0,0,0,0,0,0,0
3,4344313,NaN,3,3-pokojowe mieszkanie 56m2 + balkon Bezpośrednio,56.16,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,4407696,NaN,3,Svaran | mieszkanie 1-pokojowe | B2_D_0_1,26.96,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,3,0,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21383,4353889,NaN,3,Mieszkanie Kliny REZERWACJA do 31.05.25,43.70,605000.0,NaN,NaN,NaN,13844.39,...,0,0,0,0,0,0,0,0,0,1
21384,4373177,NaN,10,"Winogrady, os. Przyjaźni , 3 pok, Z MEBLAMI",53.00,569000.0,578950.0,1.633569e+08,NaN,10735.85,...,0,0,0,0,0,0,0,0,0,0
21385,4415128,NaN,17,"Mieszkanie, ul. Literacka",51.00,519000.0,549000.0,5.930000e+05,NaN,10176.47,...,0,0,0,0,0,0,0,0,0,0
21386,4415749,NaN,17,"Mieszkanie, ul. 1 Maja",58.00,319000.0,379000.0,4.255677e+05,NaN,5500.00,...,0,0,0,0,0,0,0,0,0,0


In [41]:
train_df_for_setup = df_balanced_for_setup.sample(frac=0.9, random_state=42)
external_holdout_df = df_balanced_for_setup.drop(train_df_for_setup.index) # Zmieńmy nazwę dla jasności
print(f"Kształt train_df_for_setup: {train_df_for_setup.shape}")
print(f"Kształt external_holdout_df: {external_holdout_df.shape}")

Kształt train_df_for_setup: (19249, 1550)
Kształt external_holdout_df: (2139, 1550)


In [42]:
df_balanced_for_setup['VoivodeshipNumber'] = df_balanced_for_setup['VoivodeshipNumber'].astype(str)
df_balanced_for_setup['CountyNumber'] = df_balanced_for_setup['CountyNumber'].astype(str)
df_balanced_for_setup['CommunityNumber'] = df_balanced_for_setup['CommunityNumber'].astype(str)
df_balanced_for_setup['KindNumber'] = df_balanced_for_setup['KindNumber'].astype(str)
df_balanced_for_setup['RegionNumber'] = df_balanced_for_setup['RegionNumber'].astype(str)
if 'StreetNumber' in df_prepared.columns:
    df_balanced_for_setup['StreetNumber'] = df_balanced_for_setup['StreetNumber'].astype(str)

In [43]:
print(f"Kształt df_prepared3 PRZED resetem indeksu: {df_balanced_for_setup.shape}")
print(f"Pierwsze 5 indeksów df_prepared3 PRZED resetem: {df_balanced_for_setup.index[:5].tolist()}") # .tolist() dla lepszego wyświetlania
if not df_balanced_for_setup.index.is_unique:
    print("UWAGA: Indeks df_prepared3 nie jest unikalny przed resetem!")
    # Jeśli indeks nie jest unikalny, może to być dodatkowy problem.
    # Rozważ df_prepared3 = df_prepared3.loc[~df_prepared3.index.duplicated(keep='first')] przed resetem.
    # Ale reset_index(drop=True) powinien i tak stworzyć nowy, unikalny indeks.

df_balanced_for_setup = df_balanced_for_setup.reset_index(drop=True)

print(f"Kształt df_prepared3 PO resecie indeksu: {df_balanced_for_setup.shape}")
print(f"Pierwsze 5 indeksów df_prepared3 PO resecie: {df_balanced_for_setup.index[:5].tolist()}")

Kształt df_prepared3 PRZED resetem indeksu: (21388, 1550)
Pierwsze 5 indeksów df_prepared3 PRZED resetem: [0, 1, 2, 3, 4]
Kształt df_prepared3 PO resecie indeksu: (21388, 1550)
Pierwsze 5 indeksów df_prepared3 PO resecie: [0, 1, 2, 3, 4]


In [44]:
# Reset indeksu dla danych przekazywanych do setup
print(f"Resetowanie indeksu dla train_df_for_setup...")
train_df_for_setup = train_df_for_setup.reset_index(drop=True)
# external_holdout_df też powinien mieć zresetowany indeks dla spójności
external_holdout_df = external_holdout_df.reset_index(drop=True)

Resetowanie indeksu dla train_df_for_setup...


In [45]:
categorical_features_initial = [
    'BuildingType', 
    'TypeOfMarket', 'OwnerType', 'Type', 'OfferFrom',
    'VoivodeshipNumber', 'CountyNumber', 'CommunityNumber', 'KindNumber',
    'RegionNumber', 'Location'
]
numeric_features_initial = [
    'Area', 'NumberOfRooms', 'Floor', 'Floors', 'CommunityScore', 'Price' 
]
date_features_initial = ['BuiltYear']

# Filtruj cechy, które faktycznie istnieją w df_prepared3
categorical_features_to_use = [col for col in categorical_features_initial if col in train_df_for_setup.columns]
numeric_features_to_use = [col for col in numeric_features_initial if col in train_df_for_setup.columns]
date_features_to_use = [col for col in date_features_initial if col in train_df_for_setup.columns]

# Definicja cech ignorowanych
ignore_features_list = [
    'SaleId', 'OriginalId', 'PortalId', 'Title', 
    'OfferPrice', 'RealPriceAfterRenovation', 'OriginalPrice',
    'PricePerSquareMeter', 'DateAddedToDatabase', 'DateAdded',
    'DateLastModification', 'DateLastRaises', 'NewestDate',
    'AvailableFrom', 'Link', 'Phone', 'MainImage', 'OtherImages',
    'NumberOfDuplicates', 'NumberOfRaises', 'NumberOfModifications',
    'IsDuplicatePriceLower', 'IsDuplicatePrivateOwner', 'Score', 'ScorePrecision',
    'NumberOfCommunityComments', 'NumberOfCommunityOpinions', 'Archive',
    'SubRegionNumber', 'EncryptedId','id', 'link',
    'StreetNumber'
]
# Filtruj ignorowane cechy, aby upewnić się, że są w DataFrame
ignore_features_list = [col for col in ignore_features_list if col in df_to_balance.columns]

transformed_exp = setup(
    data=train_df_for_setup,
    target='BuildingCondition',
    verbose=True,
    session_id=123,
    log_experiment=True,
    experiment_name=MLFLOW_EXPERIMENT_NAME,
    log_data=True,
    log_plots=True,
    fix_imbalance=False,
    categorical_features=categorical_features_to_use,
    numeric_features=numeric_features_to_use,
    date_features=date_features_to_use,
    # text_features=['opis_m'],
    ignore_features=ignore_features_list,
    ordinal_features={'BuildingType': ['Pozostałe', 'Blok', 'Apartamentowiec', 'Kamienica'],
                     },
)
best_transformed_model = transformed_exp.compare_models()
transformed_metrics_df = pull()

,Description,Value
0,Session id,123
1,Target,BuildingCondition
2,Target type,Multiclass
3,Target mapping,"AFTER_RENOVATION: 0, DEVELOPER_STATE: 1, FOR_RENOVATION: 2, GOOD: 3"
4,Original data shape,"(19249, 1550)"
5,Transformed data shape,"(19249, 1558)"
6,Transformed train set shape,"(13474, 1558)"
7,Transformed test set shape,"(5775, 1558)"
8,Ignore features,32
9,Ordinal features,1


2025/06/01 23:12:32 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.8445,0.9632,0.8445,0.8442,0.8431,0.7927,0.7935,0.9360
lightgbm,Light Gradient Boosting Machine,0.8408,0.9582,0.8408,0.8427,0.8414,0.7877,0.7880,1.7090
rf,Random Forest Classifier,0.8304,0.9610,0.8304,0.8295,0.8274,0.7739,0.7755,0.6320
ridge,Ridge Classifier,0.8297,0.0000,0.8297,0.8329,0.8305,0.7729,0.7734,0.4030
lda,Linear Discriminant Analysis,0.8279,0.0000,0.8279,0.8329,0.8297,0.7705,0.7709,1.1460
gbc,Gradient Boosting Classifier,0.8182,0.0000,0.8182,0.8198,0.8186,0.7577,0.7579,12.2380
ada,Ada Boost Classifier,0.7716,0.0000,0.7716,0.7710,0.7650,0.6955,0.6994,1.4830
dt,Decision Tree Classifier,0.7629,0.8419,0.7629,0.7619,0.7620,0.6838,0.6841,0.8560
lr,Logistic Regression,0.6966,0.0000,0.6966,0.6940,0.6946,0.5955,0.5959,8.2220
qda,Quadratic Discriminant Analysis,0.6367,0.0000,0.6367,0.6504,0.6190,0.5156,0.5291,1.8610


In [46]:
lightgbm_model = transformed_exp.create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8605,0.9634,0.8605,0.8628,0.8615,0.8140,0.8142
1,0.8450,0.9612,0.8450,0.8445,0.8441,0.7933,0.7937
2,0.8420,0.9572,0.8420,0.8432,0.8425,0.7893,0.7894
3,0.8301,0.9542,0.8301,0.8330,0.8312,0.7735,0.7737
4,0.8382,0.9605,0.8382,0.8427,0.8398,0.7842,0.7846
5,0.8070,0.9474,0.8070,0.8123,0.8091,0.7426,0.7430
6,0.8478,0.9609,0.8478,0.8492,0.8484,0.7971,0.7972
7,0.8537,0.9617,0.8537,0.8564,0.8547,0.8050,0.8052
8,0.8426,0.9551,0.8426,0.8427,0.8423,0.7901,0.7904


In [47]:
transformed_exp.dataset.head()

,Area,Price,NumberOfRooms,BuiltYear,Type,BuildingType,OfferFrom,Floor,Floors,TypeOfMarket,...,świeżym,żabka,żadnych,że,życia,życie,życiem,żłobek,żłobki,BuildingCondition
9427,79.470001,300000.0,3.0,1968-01-01,Mieszkania,NaN,Agencja,1.0,NaN,Wtórny,...,0,0,0,0,0,0,0,0,0,GOOD
1378,49.880001,NaN,3.0,2026-01-01,Mieszkania,Blok,Agencja,4.0,4.0,Pierwotny,...,0,0,0,0,0,0,0,0,0,DEVELOPER_STATE
4863,63.700001,350000.0,3.0,1988-01-01,Mieszkania,Blok,Agencja,4.0,4.0,Wtórny,...,0,0,0,0,0,0,0,0,0,FOR_RENOVATION
8588,34.000000,238000.0,1.0,NaT,Mieszkania,Blok,Osoba prywatna,NaN,NaN,Wtórny,...,0,0,0,0,0,0,0,0,0,AFTER_RENOVATION
18877,55.000000,429450.0,3.0,NaT,Mieszkania,Blok,Agencja,4.0,4.0,Wtórny,...,0,0,0,0,0,0,0,1,0,GOOD


In [48]:
print("Rozpoczynam strojenie LightGBM...")
tuned_lightgbm = transformed_exp.tune_model(lightgbm_model, optimize='F1', n_iter=10, search_library='scikit-optimize', search_algorithm='bayesian')

Rozpoczynam strojenie LightGBM...


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8494,0.9602,0.8494,0.8512,0.8500,0.7992,0.7994
1,0.8294,0.9558,0.8294,0.8288,0.8284,0.7725,0.7730
2,0.8286,0.9553,0.8286,0.8306,0.8293,0.7715,0.7717
3,0.8116,0.9533,0.8116,0.8149,0.8129,0.7488,0.7490
4,0.8300,0.9590,0.8300,0.8337,0.8313,0.7733,0.7737
5,0.7996,0.9453,0.7996,0.8040,0.8015,0.7327,0.7329
6,0.8367,0.9588,0.8367,0.8369,0.8368,0.7822,0.7823
7,0.8515,0.9601,0.8515,0.8547,0.8525,0.8020,0.8024
8,0.8255,0.9561,0.8255,0.8232,0.8237,0.7674,0.7678


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [49]:
model_do_predykcji = lightgbm_model

In [50]:
try:
    print("Próbuję sfinalizować model_do_predykcji...")
    final_lightgbm_model = transformed_exp.finalize_model(model_do_predykcji)
    print("Model sfinalizowany pomyślnie.")
    model_do_uzycia_w_skrypcie_predykcji = final_lightgbm_model
except IndexError as e:
    print(f"BŁĄD podczas finalize_model: {e}")
    print("Używam modelu przed finalizacją do dalszych predykcji.")
    model_do_uzycia_w_skrypcie_predykcji = model_do_predykcji # Wracamy do modelu przed finalizacją
except Exception as e:
    print(f"Inny BŁĄD podczas finalize_model: {e}")
    print("Używam modelu przed finalizacją do dalszych predykcji.")
    model_do_uzycia_w_skrypcie_predykcji = model_do_predykcji

Próbuję sfinalizować model_do_predykcji...
Model sfinalizowany pomyślnie.


In [51]:
if 'model_do_uzycia_w_skrypcie_predykcji' in locals():
    transformed_exp.save_model(model_do_uzycia_w_skrypcie_predykcji, "model_LGBM_stan")
    print("Model LGBM zapisany jako model_LGBM_stan.pkl")
else:
    print("Nie zdefiniowano modelu do zapisu. Sprawdź poprzednie kroki.")

Transformation Pipeline and Model Successfully Saved
Model LGBM zapisany jako model_LGBM_stan.pkl


In [52]:
from pycaret.classification import get_config
X = get_config("X")
print(X.shape)

(19249, 1517)


In [53]:
y = get_config("y")
print(y.shape)

(19249,)


In [54]:
# W Twoim przepływie, model który chcesz ocenić na holdout_df i którego metryki chcesz zalogować
# to ten, który został sfinalizowany i zapisany.
if 'model_do_uzycia_w_skrypcie_predykcji' in locals() and model_do_uzycia_w_skrypcie_predykcji is not None:
    model_do_oceny_na_holdout = model_do_uzycia_w_skrypcie_predykcji
    print(f"Używam 'model_do_uzycia_w_skrypcie_predykcji' (sfinalizowany {type(model_do_uzycia_w_skrypcie_predykcji.steps[-1][1]).__name__}) do oceny na holdout.")
elif 'final_lightgbm_model' in locals() and final_lightgbm_model is not None: # Alternatywna nazwa, jeśli tej użyłeś
    model_do_oceny_na_holdout = final_lightgbm_model
    print(f"Używam 'final_lightgbm_model' (sfinalizowany {type(final_lightgbm_model.steps[-1][1]).__name__}) do oceny na holdout.")
else:
    # Jako fallback, jeśli powyższe zawiodą, spróbujmy użyć modelu przed finalizacją
    if 'model_do_predykcji' in locals() and model_do_predykcji is not None:
         model_do_oceny_na_holdout = model_do_predykcji
         print(f"OSTRZEŻENIE: Sfinalizowany model nie znaleziony, używam 'model_do_predykcji' ({type(model_do_predykcji).__name__}) przed finalizacją.")
    elif 'lightgbm_model' in locals() and lightgbm_model is not None:
        model_do_oceny_na_holdout = lightgbm_model
        print(f"OSTRZEŻENIE: Sfinalizowany model nie znaleziony, używam 'lightgbm_model' ({type(lightgbm_model).__name__}) przed finalizacją.")
    else:
        raise NameError("Nie zdefiniowano odpowiedniego modelu (np. model_do_uzycia_w_skrypcie_predykcji, final_lightgbm_model, model_do_predykcji) do oceny na zbiorze holdout.")

# holdout_df został stworzony z df_balanced_for_setup w komórce 41.
# df_balanced_for_setup był użyty w setup().
# predict_model oczekuje danych w formie "surowej", takiej jakiej przekazano do setup().
# Więc holdout_df jest poprawnym wejściem tutaj.
print(f"Wykonywanie predykcji na zbiorze holdout_df ({external_holdout_df.shape}) używając modelu: {type(model_do_oceny_na_holdout.steps[-1][1]).__name__ if hasattr(model_do_oceny_na_holdout, 'steps') else type(model_do_oceny_na_holdout).__name__}")
predictions_on_holdout = transformed_exp.predict_model(estimator=model_do_oceny_na_holdout, data=external_holdout_df)

# Pobierz ramkę danych z metrykami dla tej predykcji
predict_holdout_df = transformed_exp.pull() # Ta linia tworzy predict_holdout_df
print("Metryki z predykcji na holdout_df:")
display(predict_holdout_df)

# Logowanie do MLflow
mlflow_exp = mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
if mlflow_exp is None:
    print(f"BŁĄD: Eksperyment MLflow '{MLFLOW_EXPERIMENT_NAME}' nie istnieje. Nie można zalogować metryk.")
else:
    runs_df = mlflow.search_runs(experiment_ids=[mlflow_exp.experiment_id], order_by=["start_time DESC"])

    if not runs_df.empty:
        # Znajdź ostatni główny przebieg (nie ten od `predict_model`)
        # Przebiegi `predict_model` są często zagnieżdżone i krótkie.
        # Chcemy zalogować do przebiegu, w którym trenowaliśmy model.
        # Zakładamy, że główny przebieg jest tym stworzonym przez `setup` lub `create_model`.
        # Możemy próbować znaleźć przebieg, który NIE ma tagu `mlflow.source.name` wskazującego na `pycaret.internal.pipeline`
        # lub po prostu wziąć ostatni, jeśli struktura jest prosta.
        
        # Logika znajdowania odpowiedniego run_id może wymagać dostosowania.
        # Dla prostoty, weźmiemy ostatni run, ale to może nie być zawsze główny run modelu.
        # Idealnie, `run_id` powinno być pobrane z obiektu `transformed_exp` po trenowaniu modelu.
        # PyCaret automatycznie loguje do aktywnego przebiegu.
        # Jeśli chcemy dodać do przebiegu stworzonego przez `setup()` lub `create_model('lightgbm')`,
        # musielibyśmy przechwycić jego run_id wcześniej.
        
        # Spróbujmy zalogować jako nowy zagnieżdżony przebieg do ostatniego głównego przebiegu
        # lub jeśli to trudne, po prostu zalogujmy do ostatniego przebiegu (co może być przebiegiem `predict_model`)
        run_id_to_log_into = runs_df['run_id'].iloc[0] # Bierzemy najnowszy run
        
        if not predict_holdout_df.empty and "Model" in predict_holdout_df.columns:
            metrics_dict = predict_holdout_df.drop(columns=["Model"]).iloc[0].to_dict()

            with mlflow.start_run(run_id=run_id_to_log_into, nested=True) as nested_run_holdout_metrics:
                print(f"Logowanie metryk holdout do MLflow run_id: {run_id_to_log_into} (nested run: {nested_run_holdout_metrics.info.run_id})")
                for key, value in metrics_dict.items():
                    if isinstance(value, (int, float, np.number)):
                        mlflow.log_metric(f"holdout_{key}", value)
                    else:
                        print(f"Pominięto logowanie metryki 'holdout_{key}' o wartości '{value}' (nie jest numeryczna).")
                mlflow.set_tag("metrics_source", "holdout_df_evaluation")
                mlflow.set_tag("model_evaluated_on_holdout", f"{type(model_do_oceny_na_holdout.steps[-1][1]).__name__ if hasattr(model_do_oceny_na_holdout, 'steps') else type(model_do_oceny_na_holdout).__name__}")

        else:
            print("BŁĄD: Ramka danych predict_holdout_df jest pusta lub nie zawiera kolumny 'Model'. Nie można zalogować metryk.")
    else:
        print(f"BŁĄD: Nie znaleziono żadnych przebiegów dla eksperymentu '{MLFLOW_EXPERIMENT_NAME}'. Nie można zalogować metryk.")

Używam 'model_do_uzycia_w_skrypcie_predykcji' (sfinalizowany LGBMClassifier) do oceny na holdout.
Wykonywanie predykcji na zbiorze holdout_df ((2139, 1550)) używając modelu: LGBMClassifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8546,0.9631,0.8546,0.8565,0.8554,0.8061,0.8062


Metryki z predykcji na holdout_df:


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8546,0.9631,0.8546,0.8565,0.8554,0.8061,0.8062


Logowanie metryk holdout do MLflow run_id: 21ec0814bf5941fe96bfd9eb81b35fdf (nested run: 21ec0814bf5941fe96bfd9eb81b35fdf)


In [55]:
mlflow_exp = mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
runs_df = mlflow.search_runs([mlflow_exp.experiment_id], order_by=["start_time DESC"])
run_id = runs_df['run_id'].values[0]
metrics = predict_holdout_df.drop(columns=["Model"]).to_dict(orient='records')[0]

with mlflow.start_run(run_id=run_id, nested=True):
    for key, value in metrics.items():
        mlflow.log_metric(key, value)

In [56]:
#predictions = transformed_exp.predict_model(best_final_model)
holdout_predictions = transformed_exp.predict_model(lightgbm_model) # Zapisz wynik predykcji
display(holdout_predictions.head()) # Wyświetl kilka pierwszych predykcji

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8452,0.9629,0.8452,0.8463,0.8457,0.7936,0.7936


,Area,Price,NumberOfRooms,BuiltYear,Type,BuildingType,OfferFrom,Floor,Floors,TypeOfMarket,...,żadnych,że,życia,życie,życiem,żłobek,żłobki,BuildingCondition,prediction_label,prediction_score
13097,34.060001,408379.0,2.0,2026-01-01,Mieszkania,Blok,Agencja,0.0,3.0,Pierwotny,...,0,0,1,0,0,1,0,DEVELOPER_STATE,DEVELOPER_STATE,0.9999
18486,49.900002,345000.0,2.0,1960-01-01,Mieszkania,Kamienica,Agencja,3.0,4.0,Wtórny,...,0,0,0,0,0,0,0,FOR_RENOVATION,FOR_RENOVATION,0.9901
12894,48.599998,225000.0,3.0,NaT,Mieszkania,Blok,Agencja,0.0,NaN,Wtórny,...,0,0,0,0,0,0,0,FOR_RENOVATION,FOR_RENOVATION,0.9283
8442,62.500000,730000.0,3.0,1990-01-01,Mieszkania,Blok,Agencja,0.0,4.0,Wtórny,...,0,0,0,0,0,0,1,FOR_RENOVATION,FOR_RENOVATION,0.9969
9572,53.680000,330000.0,2.0,1920-01-01,Mieszkania,Kamienica,Agencja,1.0,2.0,Wtórny,...,0,1,1,0,0,0,0,AFTER_RENOVATION,AFTER_RENOVATION,0.9460


In [57]:
#predicted_column = predict_holdout_df['prediction_label']
#print(predicted_column)
holdout_metrics_df = transformed_exp.pull() # Teraz pobierz metryki z tej predykcji
display(holdout_metrics_df)
predicted_column = holdout_predictions['prediction_label'] # Jeśli chcesz samą kolumnę predykcji
print(predicted_column)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8452,0.9629,0.8452,0.8463,0.8457,0.7936,0.7936


13097     DEVELOPER_STATE
18486      FOR_RENOVATION
12894      FOR_RENOVATION
8442       FOR_RENOVATION
9572     AFTER_RENOVATION
               ...       
18711                GOOD
4851     AFTER_RENOVATION
17523                GOOD
16927     DEVELOPER_STATE
19245      FOR_RENOVATION
Name: prediction_label, Length: 5775, dtype: object


In [58]:
df_last=holdout_predictions[['prediction_label','BuildingCondition', ]]

In [59]:
df_last.to_csv('data_full.csv')

In [60]:
print("Rozpoczynam dodawanie kolumny predykcji do całego zbioru data.csv...")

# 1. Wczytaj oryginalne dane
df_original_all_data = pd.read_csv('data.csv', sep=',')
print(f"Wczytano oryginalne dane: {df_original_all_data.shape[0]} wierszy.")

Rozpoczynam dodawanie kolumny predykcji do całego zbioru data.csv...
Wczytano oryginalne dane: 1133887 wierszy.


In [61]:
# 2. Utwórz kopię roboczą DataFrame do preprocessingu dla całego zbioru
df_to_preprocess_all = df_original_all_data.copy()

In [62]:
# 3. Preprocessing całego zbioru danych (musi odzwierciedlać kroki prowadzące do df_prepared3,
#    który był wejściem do setup())

# Konwersja BuiltYear na datetime (jak w komórce 8)
df_to_preprocess_all['BuiltYear'] = pd.to_datetime(df_to_preprocess_all['BuiltYear'], format='%Y', errors='coerce')

# Obsługa NaN w Description i Location przed CountVectorizer
df_to_preprocess_all['Description'] = df_to_preprocess_all['Description'].fillna('')
df_to_preprocess_all['Location'] = df_to_preprocess_all['Location'].fillna('') # Jeśli Location było używane

# Przycięcie długości Description (jak w komórce 16)
df_to_preprocess_all['Description'] = df_to_preprocess_all['Description'].str.slice(0, 3000)

# Zastosowanie nauczonego CountVectorizer (z komórki 17)
print("Transformuję kolumnę 'Description' dla całego zbioru za pomocą CountVectorizer...")
X_bow_for_all_pred = vectorizer.transform(df_to_preprocess_all["Description"])
df_bow_for_all_pred = pd.DataFrame(X_bow_for_all_pred.toarray(), columns=vectorizer.get_feature_names_out(), index=df_to_preprocess_all.index)

# Połącz cechy Bag-of-Words z resztą DataFrame
df_processed_for_all_prediction = df_to_preprocess_all.drop(columns=['Description']) # Usuń oryginalną kolumnę tekstową
df_processed_for_all_prediction = pd.concat([df_processed_for_all_prediction, df_bow_for_all_pred], axis=1)
print(f"Dane po dodaniu cech BoW: {df_processed_for_all_prediction.shape[1]} kolumn.")

# Konwersja typów kolumn numerycznych na string (jak w komórce 42)
string_conversion_cols = ['VoivodeshipNumber', 'CountyNumber', 'CommunityNumber', 'KindNumber', 'RegionNumber']
if 'StreetNumber' in df_processed_for_all_prediction.columns:
    string_conversion_cols.append('StreetNumber')

print(f"Konwertuję kolumny na string: {string_conversion_cols}")
for col in string_conversion_cols:
    if col in df_processed_for_all_prediction.columns:
        df_processed_for_all_prediction[col] = df_processed_for_all_prediction[col].astype(str)

Transformuję kolumnę 'Description' dla całego zbioru za pomocą CountVectorizer...
Dane po dodaniu cech BoW: 1550 kolumn.
Konwertuję kolumny na string: ['VoivodeshipNumber', 'CountyNumber', 'CommunityNumber', 'KindNumber', 'RegionNumber', 'StreetNumber']


In [63]:
print("Rozpoczynam predykcję dla WYPEŁNIENIA brakujących wartości w 'BuildingCondition' w data.csv...")

# 1. Wczytaj oryginalne dane
df_oryginal_do_wypelnienia_nan = pd.read_csv('data.csv', sep=',')
print(f"Wczytano oryginalne dane: {df_oryginal_do_wypelnienia_nan.shape[0]} wierszy.")
print(f"Liczba NaN w 'BuildingCondition' przed wypełnieniem: {df_oryginal_do_wypelnienia_nan['BuildingCondition'].isna().sum()}")

# 2. Utwórz kopię roboczą DataFrame do preprocessingu
df_do_preprocessingu_dla_nan = df_oryginal_do_wypelnienia_nan.copy()

# 3. Preprocessing danych (musi odzwierciedlać kroki prowadzące do danych,
#    na których trenowany był model użyty w `setup()`, np. df_balanced_for_setup lub df_prepared3)

# Konwersja BuiltYear na datetime
df_do_preprocessingu_dla_nan['BuiltYear'] = pd.to_datetime(df_do_preprocessingu_dla_nan['BuiltYear'], format='%Y', errors='coerce')

# Obsługa NaN w Description i Location przed CountVectorizer
df_do_preprocessingu_dla_nan['Description'] = df_do_preprocessingu_dla_nan['Description'].fillna('')
df_do_preprocessingu_dla_nan['Location'] = df_do_preprocessingu_dla_nan['Location'].fillna('')

# Przycięcie długości Description
df_do_preprocessingu_dla_nan['Description'] = df_do_preprocessingu_dla_nan['Description'].str.slice(0, 3000)

# Zastosowanie nauczonego CountVectorizer
print("Transformuję kolumnę 'Description' za pomocą CountVectorizer...")
X_bow_dla_nan = vectorizer.transform(df_do_preprocessingu_dla_nan["Description"])
df_bow_dla_nan = pd.DataFrame(X_bow_dla_nan.toarray(), columns=vectorizer.get_feature_names_out(), index=df_do_preprocessingu_dla_nan.index)

# Połącz cechy Bag-of-Words z resztą DataFrame
df_przetworzone_dla_nan = df_do_preprocessingu_dla_nan.drop(columns=['Description'])
df_przetworzone_dla_nan = pd.concat([df_przetworzone_dla_nan, df_bow_dla_nan], axis=1)
print(f"Dane po dodaniu cech BoW: {df_przetworzone_dla_nan.shape[1]} kolumn.")

# Konwersja typów kolumn numerycznych na string (jeśli były tak traktowane w setup)
string_conversion_cols = ['VoivodeshipNumber', 'CountyNumber', 'CommunityNumber', 'KindNumber', 'RegionNumber']
if 'StreetNumber' in df_przetworzone_dla_nan.columns:
    string_conversion_cols.append('StreetNumber')

print(f"Konwertuję kolumny na string: {string_conversion_cols}")
for col in string_conversion_cols:
    if col in df_przetworzone_dla_nan.columns:
        df_przetworzone_dla_nan[col] = df_przetworzone_dla_nan[col].astype(str)

# --- Identyfikacja wierszy z NaN i predykcja TYLKO dla nich ---
nan_indices = df_oryginal_do_wypelnienia_nan[df_oryginal_do_wypelnienia_nan['BuildingCondition'].isna()].index
print(f"Znaleziono {len(nan_indices)} wierszy z brakującym 'BuildingCondition' do wypełnienia.")

if len(nan_indices) > 0:
    # Wybierz te wiersze z przygotowanego df_przetworzone_dla_nan
    dane_do_predykcji_nan = df_przetworzone_dla_nan.loc[nan_indices].copy() # .copy() aby uniknąć SettingWithCopyWarning

    # WAŻNE: Usuń kolumnę 'BuildingCondition' z danych, na których robisz predykcję,
    # ponieważ `predict_model` zgłosi błąd, jeśli ją znajdzie z NaN-ami.
    if 'BuildingCondition' in dane_do_predykcji_nan.columns:
        print("Usuwam kolumnę 'BuildingCondition' z danych przekazywanych do predict_model (dla wierszy z NaN).")
        dane_do_predykcji_nan = dane_do_predykcji_nan.drop(columns=['BuildingCondition'])
    
    print("Wykonuję predykcje dla wierszy z brakującym 'BuildingCondition'...")
    
    # Użyj modelu, który został wybrany/sfinalizowany
    # (zakładamy, że model_do_uzycia_w_skrypcie_predykcji jest poprawnie zdefiniowany wcześniej)
    if 'model_do_uzycia_w_skrypcie_predykcji' not in locals() or model_do_uzycia_w_skrypcie_predykcji is None:
        # Próba wczytania, jeśli zmienna nie istnieje
        try:
            from pycaret.classification import load_model
            print("Zmienna 'model_do_uzycia_w_skrypcie_predykcji' nie znaleziona, próbuję wczytać 'model_LGBM_stan'...")
            model_do_uzycia_w_skrypcie_predykcji = load_model("model_LGBM_stan")
            print("Model wczytany pomyślnie.")
        except Exception as e:
            print(f"BŁĄD: Nie udało się wczytać modelu 'model_LGBM_stan': {e}")
            raise NameError("Nie zdefiniowano ani nie udało się wczytać modelu do predykcji.")


    predykcje_dla_nan_obj = transformed_exp.predict_model(
        estimator=model_do_uzycia_w_skrypcie_predykcji,
        data=dane_do_predykcji_nan
    )
    
    # 'prediction_label' to domyślna nazwa kolumny z predykcjami w PyCaret
    przewidziane_wartosci = predykcje_dla_nan_obj['prediction_label']
    
    # Wypełnij brakujące wartości w ORYGINALNYM DataFrame
    # Używamy .loc z oryginalnymi indeksami (nan_indices), aby upewnić się, że przypisujemy do właściwych miejsc
    df_oryginal_do_wypelnienia_nan.loc[nan_indices, 'BuildingCondition'] = przewidziane_wartosci.values

    print("\nZakończono wypełnianie NaN. Przykładowe wiersze, które miały NaN:")
    print(df_oryginal_do_wypelnienia_nan.loc[nan_indices, ['SaleId', 'Title', 'BuildingCondition']].head(10))
    print(f"Liczba NaN w 'BuildingCondition' PO wypełnieniu: {df_oryginal_do_wypelnienia_nan['BuildingCondition'].isna().sum()}")

else:
    print("Nie znaleziono wierszy z brakującym 'BuildingCondition' do wypełnienia.")

# Zapisz wynikowy DataFrame do nowego pliku CSV
output_filename_nan_filled = 'data_stan_nan_filled.csv'
df_oryginal_do_wypelnienia_nan.to_csv(output_filename_nan_filled, index=False, sep=',')
print(f"\nZapisano DataFrame z wypełnionymi NaN w 'BuildingCondition' do pliku: {output_filename_nan_filled}")

Rozpoczynam predykcję dla WYPEŁNIENIA brakujących wartości w 'BuildingCondition' w data.csv...
Wczytano oryginalne dane: 1133887 wierszy.
Liczba NaN w 'BuildingCondition' przed wypełnieniem: 1055766
Transformuję kolumnę 'Description' za pomocą CountVectorizer...
Dane po dodaniu cech BoW: 1550 kolumn.
Konwertuję kolumny na string: ['VoivodeshipNumber', 'CountyNumber', 'CommunityNumber', 'KindNumber', 'RegionNumber', 'StreetNumber']
Znaleziono 1055766 wierszy z brakującym 'BuildingCondition' do wypełnienia.
Usuwam kolumnę 'BuildingCondition' z danych przekazywanych do predict_model (dla wierszy z NaN).
Wykonuję predykcje dla wierszy z brakującym 'BuildingCondition'...



Zakończono wypełnianie NaN. Przykładowe wiersze, które miały NaN:
   SaleId                                              Title BuildingCondition
0      88                Mieszkanie trzypokojowe na sprzedaż  AFTER_RENOVATION
1      99  Sprzedam mieszkanie na parterze 64.8m2 Białyst...    FOR_RENOVATION
2     115      Mieszkanie bezczynszowe, 3 pokoje, 2 łazienki              GOOD
3     140                Mieszkanie trzypokojowe na sprzedaż   DEVELOPER_STATE
4     145                 3 Pokoje- 48M2-Osiedle Dziesięciny  AFTER_RENOVATION
5     159                       Mieszkanie, 87 m², Białystok  AFTER_RENOVATION
6     165               Przytulne mieszkanie blisko centrum!  AFTER_RENOVATION
7     173  **Jasne, 2 Pokojowe Mieszkanie, Widok Na Miasto**  AFTER_RENOVATION
8     179  *Bezpośrednio*PIĘKNY APARTAMENT w Centrum Lipo...  AFTER_RENOVATION
9     189                  2-pokojowe mieszkanie na sprzedaż              GOOD
Liczba NaN w 'BuildingCondition' PO wypełnieniu: 0

Zapisano Dat

In [64]:
df_nan = pd.read_csv('data_stan_nan_filled.csv', sep=',')

In [65]:
df_nan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1133887 entries, 0 to 1133886
Data columns (total 51 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   SaleId                     1133887 non-null  int64  
 1   OriginalId                 0 non-null        float64
 2   PortalId                   1133887 non-null  int64  
 3   Title                      1133887 non-null  object 
 4   Description                1125223 non-null  object 
 5   Area                       1130494 non-null  float64
 6   Price                      1080463 non-null  float64
 7   OfferPrice                 83188 non-null    float64
 8   RealPriceAfterRenovation   83188 non-null    float64
 9   OriginalPrice              110703 non-null   float64
 10  PricePerSquareMeter        1076086 non-null  float64
 11  NumberOfRooms              1119040 non-null  float64
 12  BuiltYear                  751409 non-null   float64
 13  Type        

In [66]:
df_nan.head(10)

,SaleId,OriginalId,PortalId,Title,Description,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,...,Archive,Location,VoivodeshipNumber,CountyNumber,CommunityNumber,KindNumber,RegionNumber,SubRegionNumber,StreetNumber,EncryptedId
0,88,NaN,14,Mieszkanie trzypokojowe na sprzedaż,Mieszkanie o powierzchni 73m2 znajduje się na ...,73.00,766500.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Bojary",20.0,61.0,1.0,1.0,922490.0,NaN,NaN,02TgImo1wm-Ra4zUQxoPBw==
1,99,NaN,19,Sprzedam mieszkanie na parterze 64.8m2 Białyst...,Sprzedam mieszkanie na parterze 64.8m2 w 3-pię...,64.80,540000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Antoniuk",20.0,61.0,1.0,1.0,922426.0,NaN,NaN,D-NUdVbtN3_kIW7ExgXkvA==
2,115,NaN,3,"Mieszkanie bezczynszowe, 3 pokoje, 2 łazienki",SPRZEDAŻ WYŁĄCZNIE BEZPOŚREDNIA. Agencjom nier...,51.00,540000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Bagnówka",20.0,61.0,1.0,1.0,922461.0,NaN,NaN,lmBSU1XiXLSLOq1yKSe2kA==
3,140,NaN,14,Mieszkanie trzypokojowe na sprzedaż,***Oferta bez prowizji biura i podatku PCC!***...,67.62,544000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Ul. Józefa Karola Puchal...",20.0,61.0,1.0,1.0,922410.0,NaN,43141.0,HWachf5gb6Vq-sFYH4YWnA==
4,145,NaN,3,3 Pokoje- 48M2-Osiedle Dziesięciny,Przedstawiamy na sprzedaż 3 pokojowe mieszkani...,48.00,459000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Ul. Dziesięciny",20.0,61.0,1.0,1.0,922410.0,NaN,4656.0,AWsmt4ZY0X7lNv-6fSCn9Q==
5,159,NaN,3,"Mieszkanie, 87 m², Białystok","Przestronne, jasne mieszkanie na zamkniętym os...",87.00,779000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Ul. Adama Mickiewicza",20.0,61.0,1.0,1.0,922410.0,NaN,12740.0,SEPz6jrRB1vuWWYOn99h7g==
6,165,NaN,3,Przytulne mieszkanie blisko centrum!,Na sprzedaż 2 pokojowe mieszkanie położone na ...,34.10,359000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Bojary",20.0,61.0,1.0,1.0,922490.0,NaN,16041.0,Gt5-LUpJNg-wBgBd8-hxGQ==
7,173,NaN,3,"**Jasne, 2 Pokojowe Mieszkanie, Widok Na Miasto**","Mieszkanie o powierzchni 35,7 m2 Ul. Jarzębino...",35.70,380000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok",20.0,61.0,1.0,1.0,922410.0,NaN,7290.0,a5iP5JQyibefTTLgBRU8oA==
8,179,NaN,3,*Bezpośrednio*PIĘKNY APARTAMENT w Centrum Lipo...,***REZERWACJA ***WYJĄTKOWA PROPOZYCJA DLA WYMA...,39.40,559000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Ul. Adama Mickiewicza",20.0,61.0,1.0,1.0,922410.0,NaN,12740.0,C_F7Bxl4hKQ2a2votBqs-g==
9,189,NaN,4,2-pokojowe mieszkanie na sprzedaż,Na sprzedaż 2 pokojowe mieszkanie położone na ...,34.20,354000.0,NaN,NaN,NaN,...,NaN,"Podlaskie, Białystok, Ul. Piasta",20.0,61.0,1.0,1.0,922410.0,NaN,16041.0,XylelQthYlIn_7k8orxBfQ==


In [67]:
# Nazwy plików
original_data_file = 'data.csv'
nan_filled_data_file = 'data_stan_nan_filled.csv' # Plik wygenerowany w poprzednim kroku
output_file_full = 'Data_state_out_full.csv'

print(f"Wczytuję oryginalne dane z: {original_data_file}")
df_original = pd.read_csv(original_data_file, sep=',')
print(f"Oryginalne dane wczytane. Kształt: {df_original.shape}")

print(f"Wczytuję dane z wypełnionymi NaN w BuildingCondition z: {nan_filled_data_file}")
df_nan_filled = pd.read_csv(nan_filled_data_file, sep=',')
print(f"Dane z wypełnionymi NaN wczytane. Kształt: {df_nan_filled.shape}")

# Sprawdzenie, czy oba DataFrame'y mają taką samą liczbę wierszy i najlepiej ten sam porządek
# (zakładamy, że tak jest, bo df_nan_filled powstał z df_original)
if df_original.shape[0] != df_nan_filled.shape[0]:
    print("OSTRZEŻENIE: Liczba wierszy w pliku oryginalnym i pliku z wypełnionymi NaN nie zgadza się!")
    print("Sprawdź, czy pliki są poprawne. Przerywam operację, aby uniknąć błędnego połączenia.")
    # Można by tu dodać bardziej zaawansowaną logikę łączenia po ID, jeśli kolejność nie jest gwarantowana,
    # ale na razie zakładamy, że kolejność jest zachowana.
else:
    # Tworzymy nową kolumnę 'Predict_State' w oryginalnym DataFrame
    # na podstawie (teraz już wypełnionej) kolumny 'BuildingCondition' z df_nan_filled.
    # Kolumna 'BuildingCondition' w df_nan_filled zawiera oryginalne wartości tam, gdzie były,
    # oraz przewidziane wartości tam, gdzie były NaN.
    
    print("Tworzę kolumnę 'Predict_State' w oryginalnym DataFrame...")
    df_original['Predict_State'] = df_nan_filled['BuildingCondition']

    # Wyświetlmy fragment, aby zobaczyć efekt
    print("\nPrzykładowe wiersze z oryginalną kolumną 'BuildingCondition' i nową 'Predict_State':")
    # Wybierzmy wiersze, gdzie oryginalnie było NaN, aby zobaczyć efekt wypełnienia w Predict_State
    original_nan_indices_for_display = df_original[df_original['BuildingCondition'].isna()].index[:5]
    if not original_nan_indices_for_display.empty:
        print("Przykłady, gdzie oryginalnie było NaN w BuildingCondition:")
        print(df_original.loc[original_nan_indices_for_display, ['SaleId', 'BuildingCondition', 'Predict_State']])
    
    # Przykłady, gdzie oryginalnie NIE było NaN
    original_not_nan_indices_for_display = df_original[df_original['BuildingCondition'].notna()].index[:5]
    if not original_not_nan_indices_for_display.empty:
        print("\nPrzykłady, gdzie oryginalnie BuildingCondition miało wartość:")
        print(df_original.loc[original_not_nan_indices_for_display, ['SaleId', 'BuildingCondition', 'Predict_State']])

    # Zapisz wynikowy DataFrame
    print(f"\nZapisuję połączony DataFrame do: {output_file_full}")
    df_original.to_csv(output_file_full, index=False, sep=',')
    print("Zapisano pomyślnie.")

    print(f"\nFinalny kształt pliku wyjściowego: {df_original.shape}")
    print("Pierwsze kilka wierszy pliku wyjściowego:")
    print(df_original.head())

Wczytuję oryginalne dane z: data.csv
Oryginalne dane wczytane. Kształt: (1133887, 51)
Wczytuję dane z wypełnionymi NaN w BuildingCondition z: data_stan_nan_filled.csv
Dane z wypełnionymi NaN wczytane. Kształt: (1133887, 51)
Tworzę kolumnę 'Predict_State' w oryginalnym DataFrame...

Przykładowe wiersze z oryginalną kolumną 'BuildingCondition' i nową 'Predict_State':
Przykłady, gdzie oryginalnie było NaN w BuildingCondition:
   SaleId BuildingCondition     Predict_State
0      88               NaN  AFTER_RENOVATION
1      99               NaN    FOR_RENOVATION
2     115               NaN              GOOD
3     140               NaN   DEVELOPER_STATE
4     145               NaN  AFTER_RENOVATION

Przykłady, gdzie oryginalnie BuildingCondition miało wartość:
          SaleId BuildingCondition    Predict_State
1037441  4236909   DEVELOPER_STATE  DEVELOPER_STATE
1037442  4236910   DEVELOPER_STATE  DEVELOPER_STATE
1037444  4236917   DEVELOPER_STATE  DEVELOPER_STATE
1037449  4236925   DEVELOP

In [68]:
df_nan2 = pd.read_csv('Data_state_out_full.csv', sep=',')

In [69]:
df_nan2

,SaleId,OriginalId,PortalId,Title,Description,Area,Price,OfferPrice,RealPriceAfterRenovation,OriginalPrice,...,Location,VoivodeshipNumber,CountyNumber,CommunityNumber,KindNumber,RegionNumber,SubRegionNumber,StreetNumber,EncryptedId,Predict_State
0,88,NaN,14,Mieszkanie trzypokojowe na sprzedaż,Mieszkanie o powierzchni 73m2 znajduje się na ...,73.00,766500.0,NaN,NaN,NaN,...,"Podlaskie, Białystok, Bojary",20.0,61.0,1.0,1.0,922490.0,NaN,NaN,02TgImo1wm-Ra4zUQxoPBw==,AFTER_RENOVATION
1,99,NaN,19,Sprzedam mieszkanie na parterze 64.8m2 Białyst...,Sprzedam mieszkanie na parterze 64.8m2 w 3-pię...,64.80,540000.0,NaN,NaN,NaN,...,"Podlaskie, Białystok, Antoniuk",20.0,61.0,1.0,1.0,922426.0,NaN,NaN,D-NUdVbtN3_kIW7ExgXkvA==,FOR_RENOVATION
2,115,NaN,3,"Mieszkanie bezczynszowe, 3 pokoje, 2 łazienki",SPRZEDAŻ WYŁĄCZNIE BEZPOŚREDNIA. Agencjom nier...,51.00,540000.0,NaN,NaN,NaN,...,"Podlaskie, Białystok, Bagnówka",20.0,61.0,1.0,1.0,922461.0,NaN,NaN,lmBSU1XiXLSLOq1yKSe2kA==,GOOD
3,140,NaN,14,Mieszkanie trzypokojowe na sprzedaż,***Oferta bez prowizji biura i podatku PCC!***...,67.62,544000.0,NaN,NaN,NaN,...,"Podlaskie, Białystok, Ul. Józefa Karola Puchal...",20.0,61.0,1.0,1.0,922410.0,NaN,43141.0,HWachf5gb6Vq-sFYH4YWnA==,DEVELOPER_STATE
4,145,NaN,3,3 Pokoje- 48M2-Osiedle Dziesięciny,Przedstawiamy na sprzedaż 3 pokojowe mieszkani...,48.00,459000.0,NaN,NaN,NaN,...,"Podlaskie, Białystok, Ul. Dziesięciny",20.0,61.0,1.0,1.0,922410.0,NaN,4656.0,AWsmt4ZY0X7lNv-6fSCn9Q==,AFTER_RENOVATION
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1133882,4418401,NaN,2,Mieszkanie na sprzedaż - 59 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,59.99,NaN,NaN,NaN,NaN,...,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYftiMRU9ErLfA6XOKdBw9jY=,DEVELOPER_STATE
1133883,4418402,NaN,2,Mieszkanie na sprzedaż - 41 m² - 2 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,41.23,NaN,NaN,NaN,NaN,...,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYa7D8z7yjun2jGP_aZIReN8=,DEVELOPER_STATE
1133884,4418404,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,49.09,NaN,NaN,NaN,NaN,...,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYf9eM73_U8S05NBjfLiez6E=,DEVELOPER_STATE
1133885,4418405,NaN,2,Mieszkanie na sprzedaż - 49 m² - 3 pokoje Grod...,Poznaj naszą inwestycję NA WSPÓLNEJ zlokalizow...,49.48,NaN,NaN,NaN,NaN,...,"Mazowieckie, Grodziski, Grodzisk Mazowiecki, W...",14.0,5.0,4.0,5.0,2594.0,NaN,24885.0,Ogpi4Jwj8KWDyT0rM_vFYRXVZdKE7iJxxFPu0Bq0ikc=,DEVELOPER_STATE


In [70]:
display(df_nan2[['SaleId', 'Title', 'Area', 'Price', 'BuildingCondition', 'Predict_State']].sample(25))

,SaleId,Title,Area,Price,BuildingCondition,Predict_State
875723,3830414,Rynek / pełen rozkład / ogrzewanie miejskie,67.00,899000.0,NaN,AFTER_RENOVATION
406937,2699968,Nowe 2 pokoje + obok UMEDU + GARAŻ ZA 10K + BOKS,34.00,319212.0,NaN,DEVELOPER_STATE
1080939,4326074,"Mieszkanie, Katowice, Koszutka, 36 m²",36.00,391000.0,AFTER_RENOVATION,AFTER_RENOVATION
127923,864011,Zabrze ul. Zembali Mieszkanie bezczynszowe,55.60,169000.0,NaN,FOR_RENOVATION
170802,1222678,Mieszkanie 47 m2 /3-pokoje Vetulaniego Kraków....,47.26,700000.0,NaN,AFTER_RENOVATION
899201,3890363,"Mieszkanie, Bolesława Chrobrego",46.70,499000.0,NaN,GOOD
745914,3478606,Mieszkanie na sprzedaż - 69 m² - 3 pokoje Gliw...,69.33,539000.0,NaN,DEVELOPER_STATE
663653,3258504,Mieszkanie na sprzedaż - 59 m² - 3 pokoje Lesz...,59.11,NaN,NaN,DEVELOPER_STATE
973331,4088109,Sprzedam mieszkanie 3 pokojowe z klimatyzacją,65.37,590000.0,NaN,AFTER_RENOVATION
734829,3449444,Ogródek od Ręki ! Najtaniej 3 pokoje,65.00,920000.0,NaN,DEVELOPER_STATE
